LangChain의 RAG 컴포넌트 - 문서 임베딩(Embeddings)

학습 목표
- 임베딩 모델과 벡터 데이터베이스를 효과적으로 연동할 수 있다.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
from glob import glob  

from pprint import pprint  
import json

In [ ]:
# 문서 로드
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('../data/transformer.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()

print(f'pdf 문서 개수 : {len(pdf_docs)}')

/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pdf 문서 개수 : 15


In [2]:
# 텍스트 분할
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
	chunk_size=1000,	# 청크 크기
	chunk_overlap=200,
	length_function=len, # 글자 수를 기준으로 분할
	separators=["\n\n", "\n", " ", ""],  # 구분자 - 재귀적으로 순차적으로 적용 
)

# PDF 문서를 텍스트로 분할
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 텍스트 청크 수: {len(chunks)}")

생성된 텍스트 청크 수: 52


문서 임베딩(Document Embedding)

- 개념:
	- 텍스트를 벡터(숫자배열)로 변환하는 과정
	- 문서의 의미적 특성을 수치화하여 컴퓨터가 이해하고 처리할 수 있는 형태로 변환
- 목적:
	- 텍스트 간 유사도 계산 가능
	- 벡터 데이터베이스 저장 및 검색
	- 의미 기반 문서 검색 구현
- LangChain의 임베딩 모델 종류
	- OpenAI 임베딩
	- HuggingFace 임베딩
	- Ollama 임베딩

---

1. OpenAI
	- LangChain에서 가장 널리 사용되는 임베딩 모델 중 하나
	- 주요 특징:
		1. 고품질의 임베딩 생성
		2. 다양한 언어 지원 (다국어 지원)
		3. 일관된 성능
		4. 손쉬운 통합
	- 사용시 주의사항
		1. API키 설정이 필요(환경 변수 OPENAI_API_KEY)
		2. API 사용량에 따른 비용 발생
		3. 긴 텍스트는 자동으로 분할되지 않으므로 필요시 TextSplitter를 사용

In [3]:
# 1. embedding 모델
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 모델 생성
embeddings_model=OpenAIEmbeddings(
	model='text-embedding-3-large', # 사용할 모델 이름
	dimension=None, # 원하는 임베딩 차원 수를 지정 가능 (기본값 : None)
)

# 임베딩 객체 출력
embeddings_model

/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/langchain_openai/embeddings/base.py:355: UserWarning: WARNING! dimension is not default parameter.
                    dimension was transferred to model_kwargs.
                    Please confirm that dimension is what you intended.
  warnings.warn(


OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x168d12f60>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x175d5e930>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={'dimension': None}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [4]:
# 임베딩 모델의 컨텍스트 길이 확인
embeddings_model.embedding_ctx_length

8191

In [7]:
# 임베딩 모델의 임베딩 차원 확인 - 기본값(None)
print(embeddings_model.dimensions)

None


In [8]:
# OpenAIEmbeddings 모델 생성할 때 임베딩 차원을 지정하는 예시
embeddings_model = OpenAIEmbeddings(
	model="text-embedding-3-small", # 사용할 모델 이름
	dimensions=512, # 원하는 임베딩 차원 수를 지정 가능 - 기본값: None
)

print(embeddings_model.dimensions)

512


In [ ]:
# OpenAIEmbeddings 모델 생성
embeddings_openai = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩 객체 출력
embeddings_openai

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x1753001a0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x175269a00>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [11]:
# embed_documents 사용
documents = [
	 "인공지능은 컴퓨터 과학의 한 분야입니다.",
	"머신러닝은 인공지능의 하위 분야입니다.",
	"딥러닝은 머신러닝의 한 종류입니다.",
	"자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
	"컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩
document_embeddings_openai = embeddings_openai.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_openai)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_openai[0])}")
print(document_embeddings_openai[0])

임베딩 벡터의 개수: 5
임베딩 벡터의 차원: 1536
[-0.0024314899928867817, 0.012240828014910221, -0.002483319491147995, 0.014899917878210545, 0.018433352932333946, -0.04564620926976204, -0.0032021752558648586, 0.047016315162181854, -0.018226033076643944, -0.03180091455578804, 0.0068550435826182365, 0.007021800149232149, -0.018550531938672066, -0.02711370401084423, 0.004935090895742178, -0.015819331631064415, -0.04149983078241348, 0.010041444562375546, 0.05155930295586586, -0.045898597687482834, -0.014575418084859848, -0.02767256274819374, -0.01880292035639286, -0.022570716217160225, 0.004549747798591852, -0.03991338983178139, 0.0503334179520607, 0.017730271443724632, 0.00022745922615285963, -0.02349013090133667, 0.04889119789004326, -0.01534159667789936, -0.03001616708934307, -0.06165483221411705, 0.020353306084871292, 0.03542448580265045, 0.0025666977744549513, -0.006837015971541405, -0.005128888878971338, 0.024860236793756485, 0.0074905212968587875, 0.027474258095026016, -0.0025148680433630943, 0.02610

In [12]:
# embed_query 사용 - 단일 문장을 받아서 처리할 때
embedded_query_openai = embeddings_openai.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query_openai)}")
print(embedded_query_openai)

쿼리 임베딩 벡터의 차원: 1536
[-0.02253107912838459, 0.022246355190873146, 0.0005270341061986983, 0.005661242175847292, 0.01219563465565443, -0.044834379106760025, -0.02630840428173542, 0.0357232429087162, -0.0026716506108641624, 0.0147866141051054, -0.002117627300322056, 0.0009075545240193605, -0.004873508587479591, -0.0736483484506607, 0.0090589364990592, -0.015346569009125233, -0.059753865003585815, -0.022436171770095825, 0.02239820919930935, -0.0692446306347847, -0.029212579131126404, 0.023195432499051094, -0.036843154579401016, 0.00413085613399744, 0.011123177595436573, -0.052654772996902466, 0.010762528516352177, 3.670257228804985e-06, -0.010791000910103321, -0.03410981222987175, 0.02528340183198452, -0.018896115943789482, -0.001538690528832376, -0.05895664170384407, 0.049883466213941574, -0.003516328986734152, -0.002804521471261978, -0.008399328216910362, -0.004617257975041866, 0.02279682084918022, -0.011939384043216705, 0.037982046604156494, 0.0033241407945752144, 0.035229723900556564, -

In [13]:
# 유사도 기반 검색
from langchain_community.utils.math import cosine_similarity
import numpy as np

# 쿼리와 가장 유사한 문서 찾기 함수
def find_most_similar(
		query: str,
		doc_embeddings: np.ndarray,
		embeddings_model # 기본값 제거, 명시적 전달 강제
) -> tuple[str, float]:
	"""
	쿼리와 가장 유사한 문서를 찾는 함수

	Args:
		query: 검색 쿼리 문자열
		doc_embeddings: 문자 임베딩 배열
		embeddings_model: 임베딩 모델 객체
	
	Returns:
		tuple: 가장 유사한 문서, 유사도 점수
	"""
	# 쿼리 임베딩: OpenAI 임베딩 사용
	query_embedding = embeddings_model.embed_query(query)

	# 코사인 유사도 계산
	similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

	# 가장 유사한 문서 인덱스 찾기
	most_similar_idx = np.argmax(similarities)

	# 가장 유사한 문서와 유사도 반환: 문서, 유사도
	return documents[most_similar_idx], similarities[most_similar_idx]

# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

for query in queries:
    most_similar_doc, similarity = find_most_similar(
        query, 
        document_embeddings_openai, 
        embeddings_model=embeddings_openai
        )
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

쿼리: 인공지능이란 무엇인가요?
가장 유사한 문서: 인공지능은 컴퓨터 과학의 한 분야입니다.
유사도: 0.7119

쿼리: 딥러닝과 머신러닝의 관계는 어떻게 되나요?
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도: 0.6817

쿼리: 컴퓨터가 이미지를 이해하는 방법은?
가장 유사한 문서: 컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.
유사도: 0.7052



2. Huggingface
	- LangChain에서 오픈소스 기반의 대표적인 임베딩 모델
	- 주요 특징
		1. 로컬 환경에서 실행 가능
		2. 다양한 사전학습 모델 지원
		3. 커스텀 모델 학습 및 적용 가능
		4. 무료 사용 가능 (API 비용 없음)
	- 사용시 주의사항
		1. 로컬 컴포팅 자원 필요(CPU/GPU)
		2. 초기모델 다운로드 시간 소요
		3. 메모리 사용량 고려필요
		4. transformers 라이브러리 설치 필요
	- 임베딩 백터 특성:
		1. 모델별로 다양한 차원 제공(128 - 1024)
		2. sentence-transformers 기반 구현
		3. BERT계열 모델 구조 사용
		4. 코사인 유사도 기반 검색 최적화


(1) embedding 모델
	- langchain_huggingface 설치 필요

In [15]:
from langchain_huggingface import HuggingFaceEmbeddings

# Hugging Face의 임베딩 모델 생성
embeddings_gemma = HuggingFaceEmbeddings(
	model_name="BAAI/bge-m3"
)

embeddings_gemma

HuggingFaceEmbeddings(model_name='BAAI/bge-m3', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

(2) embed_documents 사용

In [ ]:
# 문서 임베딩
document_embeddings_gemma = embeddings_gemma.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 백터의 개수: {len(document_embeddings_gemma)}")
print(f"임베딩 백터의 차원: {len(document_embeddings_gemma[0])}")
print(document_embeddings_gemma[0])

임베딩 백터의 개수: 5
임베딩 백터의 차원: 1024
[-0.0394144132733345, 0.008764924481511116, -0.01268173847347498, 0.002453141612932086, -0.008944790810346603, -0.0073836627416312695, -0.005377351772040129, -0.009055831469595432, 0.03291524946689606, 0.006045504007488489, -0.02701297216117382, -0.027740970253944397, 0.000444135075667873, 0.030136585235595703, 0.017242923378944397, 0.01709037274122238, 0.025524865835905075, -0.021856000646948814, -0.011341387405991554, -0.05702256038784981, -0.0003016542177647352, 0.013543016277253628, -0.007450132165104151, 0.018574483692646027, 0.0028947244863957167, 0.008630630560219288, -0.0007445493247359991, -0.028904207050800323, 0.02072780393064022, -0.02050061523914337, 0.008069824427366257, -0.026754220947623253, 0.003963086754083633, -0.0163038931787014, -0.07406218349933624, -0.03365037217736244, -0.02387150749564171, -0.03454999998211861, -0.03478593751788139, 0.005483087617903948, -0.05003362521529198, -0.0028036783915013075, -0.02314700558781624, -0.074911

(3) embed_query 사용

In [18]:
embedded_query = embeddings_gemma.embed_query('인공지능이란 무엇인가요?')

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query)}")
print(embedded_query)

쿼리 임베딩 벡터의 차원: 1024
[-0.03703901544213295, -0.0048380084335803986, 0.002937327604740858, -0.015514618717133999, -0.0009441781439818442, -0.04150163382291794, -0.006574476137757301, 0.011289631947875023, 0.02161405049264431, 0.0049287122674286366, -0.020340675488114357, 0.016905203461647034, -0.012874136678874493, 0.005518900230526924, 0.01498838234692812, 0.02422882243990898, 0.007369162980467081, -0.028049802407622337, -0.014939061366021633, -0.05185192450881004, -0.006705004721879959, -0.009251491166651249, -0.01698087342083454, 0.006491499021649361, 0.0529317744076252, 0.04813728109002113, -0.008069593459367752, -0.023171765729784966, 0.018143020570278168, -0.011328157037496567, -0.004240420646965504, -0.00635468540713191, -0.0022717865649610758, 0.014329486526548862, -0.03563671559095383, -0.008155811578035355, -0.011798166669905186, -0.04542405530810356, -0.04073285683989525, 0.0022139062639325857, -0.012132365256547928, 0.017896022647619247, -0.01914469711482525, -0.0419243909418

(4) 유사도 기반 검색

In [30]:
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(
        query, 
        document_embeddings_gemma, 
        embeddings_model=embeddings_gemma
     ) 
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()


쿼리: 인공지능이란 무엇인가요?
가장 유사한 문서: 인공지능은 컴퓨터 과학의 한 분야입니다.
유사도: 0.7269

쿼리: 딥러닝과 머신러닝의 관계는 어떻게 되나요?
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도: 0.7057

쿼리: 컴퓨터가 이미지를 이해하는 방법은?
가장 유사한 문서: 컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.
유사도: 0.6843



---
3. Ollama (로컬 실행 최적화)

- LangChain에서 로컬 LLM 및 임베딩 모델을 가장 손쉽게 실행할 수 있는 플랫폼입니다.
- 외부 API 전송없이 로컬 자원(GPU/CPU)만 사용하므로 데이터 보안과 비용 절감에 최적화 되어 있습니다.

- 주요 특징
	1. 완전한 로컬 실행 : 데이터가 외부로 유출되지 않아 기업 내부(On-Premise) 구축에 적합
	2. 빠른 추론 속도 : C++ 기반의 런타임과 양자화(Quantization)기술로 최저고하 됨
	3. 간편한 배포 : Docker 기반으로 모델 설치 및 실행이 매우 간단함 (Ollma Pull 모델명)

- 사용 시 주의사항
	1. 서버 실행 필수 : 백그라운드에서 Ollama serve가 실행중이어야함
	2. 리소스관리  : 고성능 모델(Large) 사용 시 충분한 RAM/VRAM 필요
	3. API 설정 : LangChain 등에서 호출 시 엔드포인트(localhost:11434) 확인 필요

- 임베딩 벡터 특성
	1. 고정 차원 : 모델별로 384 - 1024의 고정된 벡터 차원을 가짐
	2. 자동 양자화 : 원본 모델을 4비트(Q4_0)등으로 압축하여 메모리 사용량을 대폭 줄임

(1) embedding 모델
	- langchain_ollma 설치 필요

In [ ]:
from langchain_ollama import OllamaEmbeddings

# OllamaEmbeddings 모델 생성
embeddings_ollama = OllamaEmbeddings(
	model = "nomic-embed-text",						# 사용할 모델 이름
	base_url = "http://localhost:11434",		# Ollama 서버 주소
)
# embeddings_ollama = OllamaEmbeddings(model = "bge-m3")

# 임베딩 객체 출력
embeddings_ollama

OllamaEmbeddings(model='nomic-embed-text', validate_model_on_init=False, base_url='http://localhost:11434', client_kwargs={}, async_client_kwargs={}, sync_client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

(2) embed_documents 사용

In [26]:
# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩
document_embeddings_ollama = embeddings_ollama.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_ollama)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_ollama[0])}")
print(document_embeddings_ollama[0])

임베딩 벡터의 개수: 5
임베딩 벡터의 차원: 768
[0.004540187, 0.030155096, -0.18888435, -0.012818216, 0.06901435, 0.010938077, -0.017408239, -0.018095953, -0.010580853, -0.036019262, -0.05859954, 0.06212268, 0.0018659955, 0.0058077318, -0.0074306065, -0.048161488, 0.00016746107, -0.054114223, -0.05206212, 0.056111295, -0.022001913, 0.032235205, -0.023774019, -0.04446705, 0.15500093, -0.006963435, 0.037799988, 0.054320052, -0.009321075, 0.023371499, -0.0076525863, -0.01786055, -0.014387687, 0.007082765, -0.06969432, 0.0113834655, 0.0075918287, 0.019975398, 0.045739267, -0.020709965, 0.022726692, 0.049063724, 0.014890196, -0.029872257, 0.07351948, 0.014561262, 0.0322755, 0.05666565, 0.048006494, -0.037199218, -0.030006392, 0.016207973, 0.020016968, -0.02343576, 0.033161122, 0.06740819, 0.020539634, -0.02470559, 0.0008384824, -0.0035304897, 0.048678268, 0.031970374, -0.019348027, 0.03357009, 0.039992027, -0.07329384, 0.0058592944, 0.050013997, 0.045037754, 0.003420062, 0.06391212, -0.019979669, -0.01038485

(3) embed_query 사용

In [27]:
embedded_query = embeddings_ollama.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원 : {len(embedded_query)}")
print(embedded_query)

쿼리 임베딩 벡터의 차원 : 768
[-0.021755546, 0.003333297, -0.17286482, 0.014040049, 0.07018011, 0.020100843, -0.02770158, -0.01937321, -0.021918956, -0.042532798, -0.036449455, 0.050844997, 0.026821665, 0.007946613, -0.00024711445, -0.07584427, -0.006006977, -0.06304196, -0.026735019, 0.031343274, -0.02961462, 0.04690314, -0.018261634, -0.025624825, 0.1742973, -0.027013453, 0.053306185, 0.055976138, 0.008287944, 0.016290108, 0.018564982, -0.02496727, -0.0028778252, 0.019566558, -0.07991503, -0.0057435683, 0.015139678, 0.0058637364, 0.01571937, -0.017656434, 0.024816124, 0.029832441, 0.009880512, -0.04640725, 0.06399187, 0.03623761, 0.00914034, 0.058648687, 0.039167985, -0.036538124, -0.040184844, 0.025888642, -0.00619971, -0.032544952, 0.04660588, 0.07403575, 0.019911285, -0.005832169, -0.012477524, 0.03885701, 0.02405544, 0.03582993, -0.023808926, 0.04566131, 0.04089175, -0.07121171, 0.02368984, 0.039179016, 0.033632495, -0.015884506, 0.06540074, -0.029759074, 0.016280493, 0.047649488, -0.04437

(4) 유사도 기반 검색

In [29]:
# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

for query in queries:
	most_similar_doc, similarity = find_most_similar(
		query, 
		document_embeddings_ollama, 
		embeddings_model=embeddings_ollama
	)

	print(f"쿼리 : {query}")
	print(f"가장 유사한 문서 : {most_similar_doc}")
	print(f"유사도 : {similarity:.4f}")

쿼리 : 인공지능이란 무엇인가요?
가장 유사한 문서 : 머신러닝은 인공지능의 하위 분야입니다.
유사도 : 0.9247
쿼리 : 딥러닝과 머신러닝의 관계는 어떻게 되나요?
가장 유사한 문서 : 머신러닝은 인공지능의 하위 분야입니다.
유사도 : 0.9452
쿼리 : 컴퓨터가 이미지를 이해하는 방법은?
가장 유사한 문서 : 머신러닝은 인공지능의 하위 분야입니다.
유사도 : 0.9469


### 추가 필기
1. 유사도 값(소수점 숫자)의 의미
	이 숫자는 다차원 공간에 점으로 찍힌 두 문자 사이의 거리 또는 각도를 계산한 결과이다.
	- 수학적 의미 : 두 벡터(숫자배열)가 서로 얼마나 가까운 위치에 있는지를 하나의 숫자로 요약한 것이다.
	- 비즈니스적 의미 : 사용자의 질문과 데이터베이스의 문서가 얼마나 관련있는가에 대한 점수

2. 숫자가 높을 수록 유사도가 높은건가?
	이 부분은 어떤 방식을 썼느냐에 따라 다르기에 주의해야한다.
	보통 find_most_similar 매서드 내부에서는 다음 두가지 중 하나를 사용한다.

	1) 코사인 유사도(Cosine Similarity)
		- 특징 : 두 벡터 사이의 각도 측정
		- 범위 : 0 - 1 사이 수로 표현 (완전히 같으면 1)
		- 해석 : 숫자가 높을수록 1에 가까울수록 유사도가 높다.
		- OpenAI, Ollama등 대부분의 LLM 서비스에서 기본으로 권장하는 방식이다.
	
	2) 유클리드 거리(L2 Distance)
		- 특징 : 두 점 사이의 직선거리 측정
		- 범위 : 0에서 무한대
		- 해석 : 숫자가 낮을수록 (0에 가까울 수록) 유사도가 높습니다.
		- FAISS나 일부 벡터 DB에서 기본값으로 사용되기도 한다.

	3) 3개 모델 결과가 다른 이유
		동일한 질문을 해도 모델마다 숫자가 다르게 나올 것이다. 그 이유는 공간 크기와 기준이 다르기 때문 
		- OpenAI : 1536차원 공간 사용
		- HuggingFace(BGE-M3) : 1024 차원 공간 사용
		- Ollama (nomic-embed_text) : 768차원 공간 사용

3.  find_most_similar 매서드에서 코사인 유사도, 유클리드 거리 중 어떤걸 사용하는지 어떻게 알 수 있나?
	- `consine_similarity` 또는 `dot product` : 코사인 유사도 사용 (숫자가 높을 수록 좋음)
	- `numpy.linalg.norm` 또는 `spatial.distance.euclidean: 유클리드 거리를 계산 중입니다. (숫자가 낮을 수록 좋음)
	- `FAISS.similaryity_search_with_score`: LangChain 라이브러리를 쓴다면 이 함수가 점수를 반환하는데, 기본적으로 L2(유클리드)거리를 점수로 주는 경우가 많다.

	- 결국, 숫자가 1과 가까우면 cosine, 낮으면 L2?
